In [1]:
#Run if you want to autoreload your personal modules on change
import autoreload
get_ipython().run_line_magic('load_ext', 'autoreload')
get_ipython().run_line_magic('autoreload', '2')
#Next is needed in azure vm to autoreload modules in cwd
import os
pwd=os.popen("pwd").read().rstrip()
import sys
sys.path.append(pwd)
'''Train CIFAR10 with PyTorch.'''
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import numpy as np
from matplotlib import pyplot as plt
import numpy
import sklearn
import torchvision
import torchvision.transforms as transforms
import os
import argparse
#from models import *
import utils
#from utils import progress_bar
import time
from torchvision import models
import model_actions
import az_manage_proc
import load
import log
import many_inj
#import many_inj_ni
from pytorchfi_c.core import fault_injection as pfi_core
#from pytorchfi_ni.core import fault_injection as pfi_core
print("GPU available: ",torch.cuda.is_available())
print("OS: ",sys.platform)

Files already downloaded and verified
Preparing data..
Files already downloaded and verified
GPU available:  True
OS:  linux


In [2]:
num_batches = 100
c,w,h=3,32,32

In [3]:
model=model_actions.load("checkpoints/bs_128_sch_plateau/bs_128_sch_plateau_epoch_160")

In [4]:
model_actions.test(model.eval())

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


(0.8552, 35.54125779867172)

In [5]:
#debug=True? num_batches?
pfi_model = pfi_core(model.eval(), h, w, num_batches
, c=c
#,debug=True
,use_cuda=torch.cuda.is_available())

In [6]:
many_model = many_inj.many_n_inj(pfi_model,1000,min_val=0,max_val=0)

In [7]:
print(model_actions.test(many_model))

(0.8548, 35.58250993490219)


In [8]:
def test_ni():
    many_model = many_inj.many_n_inj(pfi_model,10000,min_val=0,max_val=0)
    print(model_actions.test(many_model))

In [9]:
az_manage_proc.run_and_delete(log.log_time,test_ni)

Found existing instance, use it.
Logging time
(0.8477, 37.146108746528625)
Total time: 503.306086996
Deleting VM
Found existing instance, use it.
Deleting....................